## YELP  Business table (1)



In [14]:

## YElP Database 
## business table

import json
import psycopg2
import sys

#connecting to Posgres database 

conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")


cur = conn.cursor()

#creating the business table in the database

sqlscript = '''
DROP TABLE IF EXISTS business;
	CREATE TABLE business 
        (
		business_id VARCHAR(22) NOT NULL PRIMARY KEY UNIQUE
		,name VARCHAR(255)
		,address VARCHAR(255)
		,city VARCHAR(255)
		,STATE VARCHAR(255)
		,postal_code VARCHAR(255)
		,latitude FLOAT
		,longitude FLOAT
		,stars FLOAT
		,review_count INTEGER
		,is_open INTEGER
		)
'''
cur.execute(sqlscript)

print('yelp business TABLE CREATED.')

#reading the JSON file 

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_business.json"

yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]


print('DATA LOADED: ', len(yelp_data))
counter =  0

for entry in yelp_data:
        business_id = entry['business_id']
        name = entry['name']
        address = entry['address']
        city = entry['city']
        state = entry['state']
        postal_code = entry['postal_code']
        latitude = entry['latitude']
        longitude = entry['longitude']
        stars = entry['stars']
        review_count = entry['review_count'] 
        is_open = entry['is_open']
   

        cur.execute('INSERT INTO Business (business_id ,name ,address,city ,state ,postal_code ,latitude ,longitude ,stars, review_count,is_open) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', (business_id ,name ,address,city ,state ,postal_code ,latitude ,longitude ,stars, review_count,is_open))

        
        counter =  counter +1
        counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter))
        sys.stdout.write('\r'+counter_text)
                

conn.commit()

print('   DONE.')

yelp business TABLE CREATED.
DATA LOADED:  150346
READING: 150346 INSERTING: 150346   DONE.


## YELP Category table (2)

In [1]:
## YElP Database 
## category table

import json
import psycopg2
import sys

#connecting to Posgres database 

conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")


cur = conn.cursor()

#creating the category table in the database
sqlscript = '''
DROP TABLE IF EXISTS category;
	CREATE TABLE category 
                (
		business_id VARCHAR(22),
		category VARCHAR
		)
'''
cur.execute(sqlscript)

print('yelp category TABLE CREATED.')

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_business.json"

yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]

print('DATA LOADED: ', len(yelp_data))
counter =  0
counter2 = 0
#comma separated text -> splits into a list, possible leading , trailing whitespaces removed

for entry in yelp_data:
        business_id = entry['business_id']
        categories = entry['categories']
        if categories != None:
                category_list = categories.split(",")
                category_list_stripped = [s.strip() for s in category_list]
                for entry2 in category_list_stripped:
                        categ =entry2
                        cur.execute('INSERT INTO category (business_id, category ) VALUES (%s,%s)', (business_id , categ))
                        counter2 = counter2 +1
        
        counter =  counter +1
        #if counter % 10000 ==0:
        counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter2))
        sys.stdout.write('\r'+counter_text)
                

conn.commit()
print('   DONE.')

cur.close()
conn.close()

yelp category TABLE CREATED.
DATA LOADED:  150346
READING: 150346 INSERTING: 668592   DONE.


## YELP hours table (3)

In [1]:
## YElP Database 
## hours table

from datetime import datetime, timedelta
import json
import psycopg2
import sys

#connecting to Posgres database 


conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")


cur = conn.cursor()

#dealing with the empty values to avoid key errors:

def splitter(hourstosplit):
        
        splt =[0,0]
        if hourstosplit is None:
                splt = [None,None]
        else:  splt = hourstosplit.split("-")
                
        return splt

#for converting closing 00:00 hours to 24:00 to avoid interval calculation problems:

def zero_to_24 (hour):
    if hour == '0:0' :
        hour = '24:00'
    return hour


#total open hours calculation
#sometimes a business is open after 24:00 which results in negative times as well
#this calculates the time between and corrects the negatives as well by adding 24 hours to the "till" hour if it is smaller than the "from"
def openhours (splitstring):
        if splitstring[0] is None:
               return None
        else:
                #print('incoming: ', splitstring)
                time_format = "%H:%M"
                time_converted_from = datetime.strptime((split_str[0].split(':')[0]+":"+split_str[0].split(':')[1]), time_format)
                time_converted_till = datetime.strptime((split_str[1].split(':')[0]+":"+split_str[1].split(':')[1]), time_format)
                if time_converted_from > time_converted_till:
                        time_converted_till = time_converted_till + timedelta(hours=24)
                totaltime = time_converted_till - time_converted_from
        return totaltime



sqlscript = '''
DROP TABLE IF EXISTS hours;
	CREATE TABLE hours (
		business_id VARCHAR(22)
		,hour_from time
                ,hour_till time
                ,totalhours time
		,day VARCHAR (10)
		
              )
'''
cur.execute(sqlscript)

print('yelp hours TABLE CREATED.')

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_business.json"


yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]

print('DATA LOADED: ', len(yelp_data))

counter =  0
counter2 = 0
weekday = ["Monday", "Tuesday", "Friday", "Wednesday", "Thursday", "Sunday", "Saturday"]


for entry in yelp_data:
        business_id = entry['business_id']
        hours = entry['hours']
        counter = counter + 1        
        if hours != None:
        #category_list = categories.split(",")
                #print('NOT NONE')
                for i in weekday:
                        #print(hours.get(i))
                        split_str = splitter(hours.get(i))
                        #print(i, ' from: ', split_str[0], ' till: ', split_str[1])
                        day = i
                        hour_from = split_str[0]
                        hour_till = zero_to_24(split_str[1])
                        totalhours = openhours(split_str)
                                
                        cur.execute('INSERT INTO hours (business_id, hour_from, hour_till, totalhours, day ) VALUES (%s,%s,%s,%s,%s)', (business_id, hour_from, hour_till, totalhours, day))
                        #cur.execute('INSERT INTO hours (business_id, hour, day, fromtill ) VALUES (%s,%s,%s,%s)', (business_id, to, day, 'till'))
                        counter2 = counter2 + 1
         
      
        counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter2))
        sys.stdout.write('\r'+counter_text)

conn.commit()
print('   DONE.')

cur.close()
conn.close()

yelp hours TABLE CREATED.
DATA LOADED:  150346
READING: 150346 INSERTING: 889861   DONE.


## YELP tip table (4)

In [9]:
## YElP Database 
## tip table

import json
import psycopg2
import sys

#connecting to Posgres database 


conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")


cur = conn.cursor()



sqlscript = '''
DROP TABLE IF EXISTS tip;
	CREATE TABLE tip (
		user_id VARCHAR(22)
		,business_id VARCHAR(22)
		,text VARCHAR
		,date TIMESTAMP
		,compliment_count INTEGER
		)
'''
cur.execute(sqlscript)

print('yelp tip TABLE CREATED.')

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_tip.json"


yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]


print('DATA LOADED: ', len(yelp_data))

counter =  0

for entry in yelp_data:
	
	user_id = entry['user_id']
	business_id = entry['business_id']
	text = entry['text']
	date = entry['date']
	compliment_count = entry['compliment_count']
	cur.execute('INSERT INTO tip (user_id , business_id, text, date, compliment_count ) VALUES (%s,%s,%s,%s,%s)', (user_id , business_id, text, date, compliment_count ))
	counter = counter +1
	counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter))
	sys.stdout.write('\r'+counter_text)

conn.commit()
print('   DONE.')

cur.close()
conn.close()

yelp tip TABLE CREATED.
DATA LOADED:  908915
READING: 908915 INSERTING: 908915   DONE.


## YELP checkin table (5)

(can take around 30 mins to complete)

total length of the dataset: 131 930
total rows of the table : 13 356 875    767 MBytes


In [11]:
## YElP Database 
## checkin table

import json
import psycopg2
import sys

to_load = 15000

#connecting to Posgres database 
conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")


cur = conn.cursor()

sqlscript = '''
DROP TABLE IF EXISTS checkin;
	CREATE TABLE checkin (
	business_id VARCHAR(22)
        ,date TIMESTAMP
		)
'''
cur.execute(sqlscript)

print('yelp checkin TABLE CREATED.')

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_checkin.json"


yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]

print('DATA LOADED: ', len(yelp_data), ' TO LOAD: ', to_load)

counter =  0
counter2 = 0
for entry in yelp_data:
        business_id = entry['business_id']
        date = entry['date']
        if date != None:
                date_list = date.split(",")
                for date in date_list:
                        cur.execute('INSERT INTO checkin (business_id, date ) VALUES (%s,%s)', (business_id , date))
                        counter2 = counter2 + 1

       
        counter =  counter +1
        counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter2))
        sys.stdout.write('\r'+counter_text)
        if counter == to_load:
              break
conn.commit()
print('   DONE.')

cur.close()
conn.close()

yelp checkin TABLE CREATED.
DATA LOADED:  131930
READING: 131930 INSERTING: 13356875   DONE.


## YELP user table (6)

(can take around 15 mins to complete)

In [17]:
## YElP Database 
## users table

import json
import psycopg2
import sys

#connecting to Posgres database 


conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")


cur = conn.cursor()

sqlscript = '''
DROP TABLE IF EXISTS users;
	CREATE TABLE users
		(
		user_id VARCHAR(22) NOT NULL PRIMARY KEY UNIQUE
		,name VARCHAR(255)
		,review_count INTEGER
		,yelping_since TIMESTAMP
		,useful INTEGER
		,funny INTEGER
		,cool INTEGER
		,fans INTEGER
		,elite VARCHAR
		,average_stars FLOAT
		,compliment_hot INTEGER
		,compliment_more INTEGER
		,compliment_profile INTEGER
		,compliment_cute INTEGER
		,compliment_list INTEGER
		,compliment_note INTEGER
		,compliment_plain INTEGER
		,compliment_cool INTEGER
		,compliment_funny INTEGER
		,compliment_writer INTEGER
		,compliment_photos INTEGER
		)
'''
cur.execute(sqlscript)

print('yelp users TABLE CREATED.')

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_user.json"
yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]

print('DATA LOADED: ', len(yelp_data))
counter =  0

for entry in yelp_data:
	
	user_id = entry['user_id']
	name = entry['name']
	review_count = entry['review_count']
	yelping_since = entry['yelping_since']
	useful = entry['useful']
	funny = entry['funny']
	cool = entry['cool']
	fans = entry['fans']
	elite = entry['elite']
	average_stars = entry['average_stars']
	compliment_hot = entry['compliment_hot']
	compliment_more = entry['compliment_more']
	compliment_profile = entry['compliment_profile']
	compliment_cute = entry['compliment_cute']
	compliment_list = entry['compliment_list']
	compliment_note = entry['compliment_note']
	compliment_plain = entry['compliment_plain'] 
	compliment_cool = entry['compliment_cool']
	compliment_funny = entry['compliment_funny']
	compliment_writer = entry['compliment_writer']
	compliment_photos = entry['compliment_photos']

    
        
	cur.execute('INSERT INTO users (user_id ,name,review_count ,yelping_since,useful,funny ,cool ,fans ,elite ,average_stars ,compliment_hot ,compliment_more ,compliment_profile ,compliment_cute ,compliment_list ,compliment_note ,compliment_plain ,compliment_cool , compliment_funny ,compliment_writer ,compliment_photos ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', (user_id ,name,review_count ,yelping_since ,useful,funny ,cool ,fans ,elite ,average_stars ,compliment_hot ,compliment_more ,compliment_profile ,compliment_cute ,compliment_list ,compliment_note ,compliment_plain ,compliment_cool , compliment_funny ,compliment_writer ,compliment_photos ))
		
	counter =  counter +1
	counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter))
	sys.stdout.write('\r'+counter_text)

conn.commit()
print('   DONE.')

cur.close()
conn.close()

yelp users TABLE CREATED.
DATA LOADED:  1987897
READING: 1987897 INSERTING: 1987897   DONE.


## YELP friends table (7)

(can take up to 30 hours to complete)

set limit: to_load

In [1]:
## YElP Database 
## friends table

##SET DATA TO LOAD:
to_load = 15000


import json
import psycopg2
import sys


def splitter(textsplit):
        splt =[0,0]
        if textsplit is None:
                splt = [None,None]
        else:  splt = textsplit.split("-")
        return splt


#connecting to Posgres database 

conn = psycopg2.connect(database="yelp",
                        host="localhost",
                        user="postgres",
                        password="password",
                        port="5432")
cur = conn.cursor()

sqlscript = '''
DROP TABLE IF EXISTS friends;
	CREATE TABLE friends 
        (
	user_id VARCHAR(22)
	,friend_id VARCHAR(22)
	)
'''
cur.execute(sqlscript)

print('yelp friends TABLE CREATED.')

fname = "C:/.../YELP_project/YELp/yelp_academic_dataset_user.json"
 

yelp_data = [json.loads(str_data)
        for str_data in open(fname, 'r', encoding='ISO-8859-1')]

print('DATA LOADED: ', len(yelp_data), ' TO LOAD: ', to_load)

counter =  0
counter2 = 0
for entry in yelp_data:
        user_id = entry['user_id']
        friends = entry['friends']
        
        counter = counter + 1
        if friends != None:
            friend_list = friends.split(",")
            friend_list_stripped = [s.strip() for s in friend_list]
            for friend_id in friend_list_stripped:
                cur.execute('INSERT INTO friends (user_id, friend_id ) VALUES (%s,%s)', (user_id , friend_id))
                counter2 =  counter2 +1
                
        
        counter_text = ('READING: ' + str(counter) + ' INSERTING: ' + str(counter2))
        sys.stdout.write('\r'+counter_text)
        if counter == to_load:
              break
conn.commit()
print('   DONE.')

cur.close()
conn.close()

yelp friends TABLE CREATED.
DATA LOADED:  1987897  TO LOAD:  15000
READING: 15000 INSERTING: 3709266   DONE.
